# Using Oracle Graph on 23c with Jupyter Notebooks

### Prerequisite
1. Have the BANK_ACCOUNTS.csv and BANK_TRANSFERS.csv files in the in the ./BankGraphDataset/ directory


## 1. Install necessary packages

Install python-oracledb and prettytable packages

In [56]:
## Optional
## install python-oracledb and prettytable packages in case they do not exist in the env
## use the python -m pip install approach into the current Jupyter kernel

import sys

!{sys.executable} -m pip install oracledb prettytable --upgrade

## 2. Connect to database using oracledb

The following paragraphs allow you to establish a connection to the database. Replace the username and password variables as needed.

In [39]:
import oracledb
from prettytable import PrettyTable
import csv
import getpass

user = 'GRAPHUSER'
# pw = input('Enter Password for ' + user + ' user: ')
pw = getpass.getpass('Enter Password for ' + user + ' user: ')

Enter Password for GRAPHUSER user: ········


### Connect to database

The following paragraph uses the user and pw variables from the previous paragraph to log into the database. Replaces the dsn value with the connection string for your pluggable database. It should something like the sample shown below.

In [43]:
## TODO - Edit with the connection string for your Oracle Database
oracledb.init_oracle_client()

connection = oracledb.connect(
    user=user,
    password=pw,
    dsn="""(DESCRIPTION=(CONNECT_TIMEOUT=5)(TRANSPORT_CONNECT_TIMEOUT=3)
            (RETRY_COUNT=3)(ADDRESS_LIST=(LOAD_BALANCE=on)(ADDRESS=(PROTOCOL=TCP)(HOST=<IP_Address>)
            (PORT=1521)))(CONNECT_DATA=(SERVICE_NAME=<PDB_Service_Name>)))
        """)

## 3. Create functions to execute database operations

The following paragraphs create functions to execute sql statements and commit them to the database, and execute sql queries and return them as a result set with the header values. 

In [44]:
# Function to execute a sql statement (create, insert, update, drop)  
def execute_statement(statement):
    with connection.cursor() as cursor:
        cursor.execute(statement)

In [45]:
# Function to execute a sql queries with the header and return the result set
def execute_query(query_string):
    with connection.cursor() as cursor:
        cursor.execute(query_string)
        columns = [col[0] for col in cursor.description]
        return [columns, cursor.fetchall()]

In [54]:
# Function to verify if a table exists for the current user
def table_exists(tablename):
    with connection.cursor() as cursor:
        cursor.execute("""SELECT TABLE_NAME FROM USER_TABLES WHERE TABLE_NAME = :x1""", [tablename])
        return cursor.fetchone() is not None
        

## 4. Load dataset

Load the bank graph dataset using the oracledb library

In [55]:
# Create BANK_ACCOUNTS table if it does not already exist
if not table_exists('BANK_ACCOUNTS'):
    s = '''CREATE TABLE BANK_ACCOUNTS (
                ID              NUMBER,
                NAME            VARCHAR(400),
                BALANCE         NUMBER(20,2)
            )'''
    execute_statement(s)
    
    # Load BANK_ACCOUNTS data from csv
    with open('./BankGraphDataset/BANK_ACCOUNTS.csv', newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        bank_acct_data = [row for row in csvreader]
        with connection.cursor() as cursor:
            cursor.executemany('''INSERT INTO BANK_ACCOUNTS VALUES(:x1, :x2, :x3)''', bank_acct_data[1:])

In [8]:
# Create BANK_TRANSFERS table if it does not already exist
if not table_exists('BANK_TRANSFERS'):
    s = '''CREATE TABLE BANK_TRANSFERS (
                TXN_ID          NUMBER,
                SRC_ACCT_ID     NUMBER,
                DST_ACCT_ID     NUMBER,
                DESCRIPTION     VARCHAR(400),
                AMOUNT          NUMBER
            )'''
    execute_statement(s)
    
    # Load BANK_TRANSFERS data from csv
    with open('./BankGraphDataset/BANK_TRANSFERS.csv', newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        bank_transfer_data = [row for row in csvreader]
        with connection.cursor() as cursor:
            cursor.executemany('''INSERT INTO BANK_TRANSFERS VALUES(:x1, :x2, :x3, :x4, :x5)''', bank_transfer_data[1:])

In [18]:
# Add constraints
constraints = ['ALTER TABLE BANK_ACCOUNTS ADD PRIMARY KEY (ID)', 'ALTER TABLE BANK_TRANSFERS ADD PRIMARY KEY (TXN_ID)', 'ALTER TABLE BANK_TRANSFERS MODIFY SRC_ACCT_ID REFERENCES BANK_ACCOUNTS (ID)', 'ALTER TABLE BANK_TRANSFERS MODIFY DST_ACCT_ID REFERENCES BANK_ACCOUNTS (ID)']

for constraint in constraints: 
    execute_statement(constraint)
    
    

## 5. Create property graph

Create the property graph by using the execute_statement function.

In [22]:
# Create a property graph view on bank_accounts and bank_transfers
statement = """CREATE PROPERTY GRAPH bank_graph 
    VERTEX TABLES (
        BANK_ACCOUNTS
        KEY (ID)
        PROPERTIES (ID, Name, Balance) 
    )
    EDGE TABLES (
        BANK_TRANSFERS 
        KEY (TXN_ID) 
        SOURCE KEY (src_acct_id) REFERENCES BANK_ACCOUNTS(ID)
        DESTINATION KEY (dst_acct_id) REFERENCES BANK_ACCOUNTS(ID)
        PROPERTIES (src_acct_id, dst_acct_id, amount)
    ) """
execute_statement(statement)

## 6. Run SQL queries to gather information

In [23]:
# This query shows the graphs available for the current user
query = """SELECT * FROM USER_PROPERTY_GRAPHS"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+--------------------+------------+--------------------+----------+
|     GRAPH_NAME     | GRAPH_MODE | ALLOWS_MIXED_TYPES | INMEMORY |
+--------------------+------------+--------------------+----------+
|   MY_JSON_GRAPH    |  TRUSTED   |         NO         |    NO    |
|      MY_GRAPH      |  TRUSTED   |         NO         |    NO    |
| MY_JSON_PROP_GRAPH |  TRUSTED   |         NO         |    NO    |
|  BANK_TRANSFERS1   |  TRUSTED   |         NO         |    NO    |
|     BANK_GRAPH     |  TRUSTED   |         NO         |    NO    |
+--------------------+------------+--------------------+----------+


In [24]:
# This query shows the DDL for the BANK_GRAPH graph 
query = """SELECT DBMS_METADATA.GET_DDL('PROPERTY_GRAPH', 'BANK_GRAPH') FROM DUAL"""
rs = execute_query(query)
for obj in rs[1]:
    print(str(obj[0]))


  CREATE PROPERTY GRAPH "GRAPHUSER"."BANK_GRAPH" 
  VERTEX TABLES (
   "GRAPHUSER"."BANK_ACCOUNTS" AS "BANK_ACCOUNTS" KEY ("ID")
      PROPERTIES ("ID", "NAME", "BALANCE") )
  EDGE TABLES (
   "GRAPHUSER"."BANK_TRANSFERS" AS "BANK_TRANSFERS" KEY ("TXN_ID")
      SOURCE KEY("SRC_ACCT_ID") REFERENCES BANK_ACCOUNTS ("ID")
      DESTINATION KEY("DST_ACCT_ID") REFERENCES BANK_ACCOUNTS ("ID")
     PROPERTIES ("SRC_ACCT_ID", "DST_ACCT_ID", "AMOUNT") )
  OPTIONS (TRUSTED MODE, DISALLOW MIXED PROPERTY TYPES)


In [25]:
# This query shows the elements for the BANK_TRANSFERS graph
query = """SELECT * FROM USER_PG_ELEMENTS WHERE GRAPH_NAME='BANK_GRAPH'"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+------------+----------------+--------------+--------------+----------------+
| GRAPH_NAME |  ELEMENT_NAME  | ELEMENT_KIND | OBJECT_OWNER |  OBJECT_NAME   |
+------------+----------------+--------------+--------------+----------------+
| BANK_GRAPH | BANK_TRANSFERS |     EDGE     |  GRAPHUSER   | BANK_TRANSFERS |
| BANK_GRAPH | BANK_ACCOUNTS  |    VERTEX    |  GRAPHUSER   | BANK_ACCOUNTS  |
+------------+----------------+--------------+--------------+----------------+


In [26]:
# This query shoes the labels and properties available in the BANK_TRANSFERS graph
query = """SELECT GRAPH_NAME, LABEL_NAME, PROPERTY_NAME, DATA_LENGTH, DATA_CHAR_LENGTH, PROPERTY_ORDER
            FROM USER_PG_LABEL_PROPERTIES WHERE GRAPH_NAME='BANK_GRAPH'"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+------------+----------------+---------------+-------------+------------------+----------------+
| GRAPH_NAME |   LABEL_NAME   | PROPERTY_NAME | DATA_LENGTH | DATA_CHAR_LENGTH | PROPERTY_ORDER |
+------------+----------------+---------------+-------------+------------------+----------------+
| BANK_GRAPH | BANK_TRANSFERS |  SRC_ACCT_ID  |      22     |        0         |       1        |
| BANK_GRAPH | BANK_TRANSFERS |  DST_ACCT_ID  |      22     |        0         |       2        |
| BANK_GRAPH | BANK_TRANSFERS |     AMOUNT    |      22     |        0         |       3        |
| BANK_GRAPH | BANK_ACCOUNTS  |      NAME     |     400     |       400        |       2        |
| BANK_GRAPH | BANK_ACCOUNTS  |    BALANCE    |      22     |        0         |       3        |
| BANK_GRAPH | BANK_ACCOUNTS  |       ID      |      22     |        0         |       1        |
+------------+----------------+---------------+-------------+------------------+----------------+


## 7. Use SQL/PGQ queries to query the BANK_TRANSFERS graph
**SQL/PGQ is a property graph query extension included in the SQL:2023 standard.**
It defines the CREATE PROPERTY GRAPH DDL, the GRAPH_TABLE() operator to query a graph, and the MATCH clause to specify the pattern to look for in the graph

In [29]:
# Find the top 10 accounts by incoming transfers 
query = """SELECT ACCT_ID, COUNT(1) AS NUM_TRANSFERS
    FROM GRAPH_TABLE ( BANK_GRAPH
    MATCH (SRC) - [IS BANK_TRANSFERS] -> (DST)
    COLUMNS ( DST.ID AS ACCT_ID )
    ) GROUP BY ACCT_ID ORDER BY NUM_TRANSFERS DESC FETCH FIRST 10 ROWS ONLY"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+---------+---------------+
| ACCT_ID | NUM_TRANSFERS |
+---------+---------------+
|   387   |       39      |
|   934   |       39      |
|   135   |       36      |
|   534   |       32      |
|   380   |       31      |
|   330   |       30      |
|   406   |       28      |
|   746   |       28      |
|   920   |       26      |
|   259   |       26      |
+---------+---------------+


In [30]:
# Find the top 10 accounts in the middle of a 2-hop chain of transfers
query = """SELECT ACCT_ID, COUNT(1) AS NUM_IN_MIDDLE
    FROM GRAPH_TABLE ( BANK_GRAPH
    MATCH (SRC) - [IS BANK_TRANSFERS] -> (VIA) - [IS BANK_TRANSFERS] -> (DST)
    COLUMNS ( VIA.ID AS ACCT_ID )
    ) GROUP BY ACCT_ID ORDER BY NUM_IN_MIDDLE DESC FETCH FIRST 10 ROWS ONLY"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+---------+---------------+
| ACCT_ID | NUM_IN_MIDDLE |
+---------+---------------+
|   387   |      195      |
|   934   |      195      |
|   135   |      180      |
|   534   |      160      |
|   380   |      155      |
|   330   |      150      |
|   406   |      140      |
|   746   |      140      |
|   920   |      130      |
|   259   |      130      |
+---------+---------------+


In [31]:
# List accounts that received a transfer from account 387 in 1, 2, or 3 hops
query = """SELECT ACCOUNT_ID1, ACCOUNT_ID2 FROM GRAPH_TABLE(BANK_GRAPH
    MATCH (V1)-[IS BANK_TRANSFERS]->{1,3}(V2)
    WHERE V1.ID = 387
    COLUMNS (V1.ID AS ACCOUNT_ID1, V2.ID AS ACCOUNT_ID2))"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+-------------+-------------+
| ACCOUNT_ID1 | ACCOUNT_ID2 |
+-------------+-------------+
|     387     |     577     |
|     387     |     867     |
|     387     |     998     |
|     387     |     188     |
|     387     |     374     |
|     387     |     926     |
|     387     |      63     |
|     387     |     183     |
|     387     |     463     |
|     387     |     581     |
|     387     |     678     |
|     387     |     675     |
|     387     |     970     |
|     387     |     693     |
|     387     |     439     |
|     387     |     814     |
|     387     |     871     |
|     387     |     548     |
|     387     |     558     |
|     387     |     781     |
|     387     |     783     |
|     387     |     479     |
|     387     |      38     |
|     387     |     662     |
|     387     |     276     |
|     387     |     897     |
|     387     |     624     |
|     387     |     297     |
|     387     |     297     |
|     387     |     406     |
|     387 

In [32]:
# Check if there are any 3-hop (triangles) transfers that start and end at the same account
query = """SELECT ACCT_ID, COUNT(1) AS NUM_TRIANGLES
    FROM GRAPH_TABLE (BANK_GRAPH
    MATCH (SRC) - []->{3} (SRC)
    COLUMNS (SRC.ID AS ACCT_ID)
    ) GROUP BY ACCT_ID ORDER BY NUM_TRIANGLES DESC"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+---------+---------------+
| ACCT_ID | NUM_TRIANGLES |
+---------+---------------+
|   751   |       3       |
|   534   |       3       |
|   918   |       3       |
|   359   |       3       |
|   781   |       2       |
|   677   |       2       |
|   463   |       2       |
|   203   |       2       |
|   119   |       2       |
|   673   |       2       |
|   218   |       2       |
|   651   |       2       |
|   717   |       2       |
|   366   |       1       |
|   804   |       1       |
|   940   |       1       |
|   317   |       1       |
|   782   |       1       |
|   813   |       1       |
|   306   |       1       |
|    82   |       1       |
|   289   |       1       |
|   134   |       1       |
|   926   |       1       |
|   579   |       1       |
|   580   |       1       |
|    88   |       1       |
|    22   |       1       |
|   340   |       1       |
|   625   |       1       |
|   337   |       1       |
|   412   |       1       |
|   404   |       1 

In [33]:
# Check if there are any 4-hop transfers that start and end at the same account
query = """SELECT ACCT_ID, COUNT(1) AS NUM_4HOP_CHAINS
    FROM GRAPH_TABLE (BANK_GRAPH
    MATCH (SRC) - []->{4} (SRC)
    COLUMNS (SRC.ID AS ACCT_ID)
    ) GROUP BY ACCT_ID ORDER BY NUM_4HOP_CHAINS DESC"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+---------+-----------------+
| ACCT_ID | NUM_4HOP_CHAINS |
+---------+-----------------+
|   397   |        8        |
|   387   |        7        |
|   579   |        7        |
|   499   |        6        |
|   559   |        6        |
|   801   |        6        |
|    66   |        5        |
|   640   |        5        |
|   716   |        5        |
|   867   |        5        |
|   276   |        5        |
|   560   |        5        |
|   406   |        5        |
|   325   |        5        |
|   359   |        4        |
|   412   |        4        |
|   877   |        4        |
|   524   |        4        |
|   607   |        4        |
|   263   |        4        |
|   734   |        4        |
|   231   |        4        |
|   458   |        4        |
|   918   |        4        |
|   259   |        4        |
|   382   |        4        |
|   821   |        4        |
|   352   |        4        |
|   293   |        4        |
|    13   |        4        |
|   650   

In [34]:
# Check if there are any 5-hop transfers that start and end at the same account
query = """SELECT ACCT_ID, COUNT(1) AS NUM_5HOP_CHAINS
FROM GRAPH_TABLE (BANK_GRAPH
MATCH (SRC) - []->{5} (SRC)
COLUMNS (SRC.ID AS ACCT_ID)
) GROUP BY ACCT_ID ORDER BY NUM_5HOP_CHAINS DESC"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+---------+-----------------+
| ACCT_ID | NUM_5HOP_CHAINS |
+---------+-----------------+
|   135   |        35       |
|   934   |        27       |
|   387   |        27       |
|    13   |        23       |
|   640   |        23       |
|   458   |        22       |
|   559   |        19       |
|   352   |        19       |
|   406   |        18       |
|   998   |        17       |
|   642   |        17       |
|    57   |        16       |
|   499   |        16       |
|   738   |        16       |
|   228   |        16       |
|   222   |        16       |
|   380   |        15       |
|   317   |        15       |
|   560   |        15       |
|   644   |        15       |
|   904   |        15       |
|    4    |        15       |
|   534   |        14       |
|    71   |        14       |
|   856   |        14       |
|   716   |        14       |
|   123   |        14       |
|   448   |        14       |
|   921   |        14       |
|   241   |        14       |
|   325   

In [35]:
# List some (any 10) accounts which had a 3 to 5 hop circular payment chain 
query = """SELECT DISTINCT(ACCOUNT_ID)
FROM GRAPH_TABLE(BANK_GRAPH
   MATCH (V1)-[IS BANK_TRANSFERS]->{3,5}(V1)
    COLUMNS (V1.ID AS ACCOUNT_ID)
) FETCH FIRST 10 ROWS ONLY"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+------------+
| ACCOUNT_ID |
+------------+
|    813     |
|    781     |
|    579     |
|    359     |
|     6      |
|    753     |
|    534     |
|    774     |
|    685     |
|    535     |
+------------+


In [37]:
# Query accounts by number of 3 to 5 hops cycles in descending order. Show top 10. 
query = """SELECT DISTINCT(ACCOUNT_ID), COUNT(1) AS NUM_CYCLES FROM GRAPH_TABLE(BANK_GRAPH
    MATCH (V1)-[IS BANK_TRANSFERS]->{3, 5}(V1)
    COLUMNS (V1.ID AS ACCOUNT_ID) )
    GROUP BY ACCOUNT_ID ORDER BY NUM_CYCLES DESC FETCH FIRST 10 ROWS ONLY"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+------------+------------+
| ACCOUNT_ID | NUM_CYCLES |
+------------+------------+
|    135     |     37     |
|    387     |     34     |
|    934     |     30     |
|    640     |     28     |
|    458     |     27     |
|     13     |     27     |
|    559     |     25     |
|    352     |     23     |
|    406     |     23     |
|    499     |     22     |
+------------+------------+


In [59]:
# List all accounts in a 5 hop (or 6 hop) circular chain 
query = """SELECT START_ACCT, FIRST_ACCT, SECOND_ACCT, THIRD_ACCT, FOURTH_ACCT, FIFTH_ACCT, START_ACCT AS END_IN_ACCT
            FROM GRAPH_TABLE(BANK_GRAPH
            MATCH (SRC) -[IS BANK_TRANSFERS]->(F) -[IS BANK_TRANSFERS]-> (S)
            -[IS BANK_TRANSFERS]-> (T) -[IS BANK_TRANSFERS]-> (FO)
            -[IS BANK_TRANSFERS]-> (FI) -[IS BANK_TRANSFERS]->(SRC)
        WHERE SRC.ID=934
        COLUMNS (
        SRC.ID AS START_ACCT,
        F.ID AS FIRST_ACCT,
        S.ID AS SECOND_ACCT,
        T.ID AS THIRD_ACCT,
        FO.ID AS FOURTH_ACCT,
        FI.ID AS FIFTH_ACCT)
        )"""
rs = execute_query(query)
table = PrettyTable()
table.field_names = rs[0]

for row in rs[1]:
    table.add_row(row)
    
print(table)

+------------+------------+-------------+------------+-------------+------------+-------------+
| START_ACCT | FIRST_ACCT | SECOND_ACCT | THIRD_ACCT | FOURTH_ACCT | FIFTH_ACCT | END_IN_ACCT |
+------------+------------+-------------+------------+-------------+------------+-------------+
|    934     |    369     |     799     |    842     |     226     |    487     |     934     |
|    934     |    597     |     348     |    177     |     242     |    135     |     934     |
|    934     |    651     |     824     |    420     |     245     |    942     |     934     |
|    934     |    651     |     944     |    181     |      23     |    302     |     934     |
|    934     |    651     |     944     |    181     |      23     |    528     |     934     |
|    934     |    651     |     272     |    607     |      60     |    135     |     934     |
|    934     |    651     |      13     |    749     |      60     |    135     |     934     |
|    934     |    406     |      66     

## 8. Commit and close connection 

In [ ]:
## Optional: commit if you want data to persist
connection.commit()

In [ ]:
connection.close()